# Association between Fertility & Local Municipal Characteristics in Denmark 

### Imports

In [20]:
import geopandas as gpd # for geospatial data handling
import pandas as pd
import contextily as cx # for plotting
import matplotlib.pyplot as plt # for plotting
from pyproj import CRS # for more advanced CRS modifications and transformations

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

In [44]:
# load the saved GeoDataFrame from file
gdf = gpd.read_file('Data/gdf.geojson')

## Inspecting spatial data and EDA

## Spatial Weights

In [31]:
import seaborn as sns
sns.set_theme()
from pysal.lib import weights
from libpysal.io import open as psopen
import numpy as np
import matplotlib.pyplot as plt
from splot.libpysal import plot_spatial_weights

In [35]:
gdf.info

<bound method DataFrame.info of     municipality_nm fer_rate live_birth  edu_att market_val  \
0         København  1 563.9     10 285  501 212  2 399 736   
1     Frederiksberg  1 787.8      1 534   74 788  3 435 510   
2            Dragør  2 510.9        146    9 074  4 014 690   
3            Tårnby  2 056.7        473   28 727  2 823 345   
4       Albertslund  1 608.7        274   18 664  2 931 561   
..              ...      ...        ...      ...        ...   
93            Morsø  2 080.7        180   13 103  1 321 820   
94           Rebild  2 400.5        380   20 166  1 997 252   
95          Thisted  2 063.8        410   28 630  1 422 277   
96  Vesthimmerlands  1 824.4        314   24 005  1 495 512   
97          Aalborg  1 604.3      2 488  158 831  2 042 461   

                                             geometry  
0   MULTIPOLYGON (((672902.450 6143708.870, 672880...  
1   MULTIPOLYGON (((654675.250 6154223.210, 654636...  
2   MULTIPOLYGON (((563379.450 6175110.360,

In [32]:
w_queen = weights.Queen.from_dataframe(gdf, idVariable="fer_rate")

ValueError: The argument to the ids parameter contains duplicate entries.